# <center> Battle Of the Neighborhoods: Toronto and Manhattan<center>

## Introduction

Two of the largest cities in the world, Manhattan and Toronto, are constantly receiving attention from potential businessmen/women. As many can simply physically fly over to the cities to see what they offer, it proves to be inconvenient to do so, especially in times like during the current pandemic. The project developed here will allow these people to view business in the areas of both cities with location data and visual aids to provide further details to them in order to assist in their decision. This project removes the need for constant research or physical travel by conveniently giving all the details in a well designed manner.

## Data

We will begin by scraping the Wikipedia pages at https://en.wikipedia.org/wiki/List_of_Manhattan_neighborhoods and also at https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M . The Manhattan table will be mainly used for its "Name of the neighborhood" column while the Canada table will be used for "Postal Code", "Borough", and "Neighbourhood". We will also be using the ArcGIS API in order to develop longitude and latitude coordinates for each neighborhood and will then be using Foursquare location data to discover local business within each neighborhood. Neighbourhood, Neighbourhood Latitude , Neighbourhood Longitude , Venue , Venue Latitude , Venue Longitude, and Venue Category will all be collected using the Foursquare location data. We then cluster the neighborhoods based on similar values and present the information.

## Methodology

Here we import all the required packages and tools needed for data processing, visualization, and collection.

In [137]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!pip install geopy
from IPython.display import Image 
from IPython.core.display import HTML 
import requests
import pandas as pd
from datetime import datetime
import numpy as np
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

!pip install geocoder
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
!pip install geopandas
!pip install geopy

Folium installed
Libraries imported.


* Pandas : To collect and manipulate data in JSON and HTMl and then data analysis
* requests : Handle http requests
* matplotlib : Detailing the generated maps
* folium : Generating maps
* sklearn : To import Kmeans which is the machine learning model that we are using.
* geocoder: in order to convert location into longitude and latitude values.

### Here we need to scrape the data from the wikipedia sites in order to retrieve the information for each city.

In [138]:
url_ny = "https://en.wikipedia.org/wiki/List_of_Manhattan_neighborhoods"
wiki_ny_url = requests.get(url_ny)
wiki_ny_data = pd.read_html(wiki_ny_url.text)
wiki_ny_data = wiki_ny_data[1]
wiki_ny_data

,Name of the neighborhood,Limits south to north and east to west
0,Columbus Circle,59th Street and 8th Avenue
1,Sutton Place,53rd to 59th Streets; 1st Avenue to Sutton Place
2,Rockefeller Center,49th to 51st Streets; 5th to 6th Avenues
3,Diamond District,47th Street from 5th to 6th Avenues
4,Theater District,42nd to 53rd Streets; 6th to 8th Avenues
5,Turtle Bay,42nd to 53rd Streets; East River to Lexington ...
6,Midtown East,42nd to 59th Streets; East River to 5th Avenue
7,Midtown,40th to 59th Streets; 3rd to 9th Avenues
8,Tudor City,40th to 43rd Streets; 1st to 2nd Avenues
9,Little Brazil,46th Street from 5th to 6th Avenues


In [139]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
wiki_toronto_url = requests.get(url)
wiki_toronto_url

wiki_toronto_data = pd.read_html(wiki_toronto_url.text)
wiki_toronto_data

[    Postal Code           Borough  \
 0           M1A      Not assigned   
 1           M2A      Not assigned   
 2           M3A        North York   
 3           M4A        North York   
 4           M5A  Downtown Toronto   
 ..          ...               ...   
 175         M5Z      Not assigned   
 176         M6Z      Not assigned   
 177         M7Z      Not assigned   
 178         M8Z         Etobicoke   
 179         M9Z      Not assigned   
 
                                          Neighbourhood  
 0                                         Not assigned  
 1                                         Not assigned  
 2                                            Parkwoods  
 3                                     Victoria Village  
 4                            Regent Park, Harbourfront  
 ..                                                 ...  
 175                                       Not assigned  
 176                                       Not assigned  
 177                

### Here we will remove the spaces with underscores in the NY data feature names.

In [140]:
wiki_ny_data.rename(columns=lambda x: x.strip().replace(" ", "_"), inplace=True)

In [141]:
wiki_ny_data

,Name_of_the_neighborhood,Limits_south_to_north_and_east_to_west
0,Columbus Circle,59th Street and 8th Avenue
1,Sutton Place,53rd to 59th Streets; 1st Avenue to Sutton Place
2,Rockefeller Center,49th to 51st Streets; 5th to 6th Avenues
3,Diamond District,47th Street from 5th to 6th Avenues
4,Theater District,42nd to 53rd Streets; 6th to 8th Avenues
5,Turtle Bay,42nd to 53rd Streets; East River to Lexington ...
6,Midtown East,42nd to 59th Streets; East River to 5th Avenue
7,Midtown,40th to 59th Streets; 3rd to 9th Avenues
8,Tudor City,40th to 43rd Streets; 1st to 2nd Avenues
9,Little Brazil,46th Street from 5th to 6th Avenues


### In this section, we drop blank/null values from the Canada information as well as drop the index feature entirely. We will also group by postal code.

In [142]:
wiki_toronto_data = wiki_toronto_data[0]
wiki_toronto_data

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [143]:
df = wiki_toronto_data[wiki_toronto_data["Borough"] != "Not assigned"]
df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [144]:
df = df.groupby(['Postal Code']).head()
df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [145]:
df.Neighbourhood.str.count("Not assigned").sum()

0

In [146]:
df = df.reset_index()
df

,index,Postal Code,Borough,Neighbourhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...,...
98,160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,165,M4Y,Downtown Toronto,Church and Wellesley
100,168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [147]:
df.drop(['index'], axis = 'columns', inplace = True)
df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### Here we will get the longitude and latitude coordinates for Canada's data.

In [148]:
data = pd.read_csv("https://cocl.us/Geospatial_data")
data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


### Afterwards, we join the two datasets in order to easily locate each neighborhood in later use.

In [149]:
combined_toronto_data = df.join(data.set_index('Postal Code'), on='Postal Code', how='inner')
combined_toronto_data

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


### Here we drop the Limits_south_to_north_and_east_to_west feature from the NY dataset.

In [150]:
df1 = wiki_ny_data.drop( [ wiki_ny_data.columns[1]], axis=1)

In [207]:
df1 = df1.drop([18,19])

### We will now filter the Canada data for only Toronto neighborhoods.

In [209]:
combined_toronto_data = combined_toronto_data[combined_toronto_data['Borough'].str.contains('Toronto')]

In [210]:
combined_toronto_data

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


### We now need location data for New York and toronto in order to find local businesses.

In [211]:
from arcgis.geocoding import geocode
from arcgis.gis import GIS
gis = GIS()

In [212]:
address_toronto= 'Toronto, Ontario'
geolocator_toronto = Nominatim(user_agent="toronto_explorer")
locator = Nominatim(user_agent="myGeocoder")
location_toronto = geolocator_toronto.geocode(address_toronto)
latitude_toronto = location_toronto.latitude
longitude_toronto = location_toronto.longitude

In [213]:
address_ny= 'Manhattan, New York, USA'
geolocator_ny = Nominatim(user_agent="ny_explorer")
locator = Nominatim(user_agent="myGeocoder")
location_ny = geolocator_ny.geocode(address_ny)
latitude_ny = location_ny.latitude
longitude_ny = location_ny.longitude

### Since we already have longitude and latitude for Toronto, we need to use geopy in order to get that data for NY.

In [214]:
from geopy.extra.rate_limiter import RateLimiter

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df1['location'] = df1['Name_of_the_neighborhood'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df1['point'] = df1['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df1[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df1['point'].tolist(), index=df1.index)

In [215]:
df1

,Name_of_the_neighborhood,location,point,latitude,longitude,altitude
0,Columbus Circle,"(Columbus Circle, Broadway, Columbus Circle, M...","(40.76808045, -73.98189744682783, 0.0)",40.768080,-73.981897,0.0
1,Sutton Place,"(Sutton Place, Bull Lane, Sutton Green, Woking...","(51.27196705, -0.5510324033859777, 0.0)",51.271967,-0.551032,0.0
2,Rockefeller Center,"(Rockefeller Center, 45, Rockefeller Plaza, Mi...","(40.75925755, -73.97995686116377, 0.0)",40.759258,-73.979957,0.0
3,Diamond District,"(Diamond District, West 47th Street, Diamond D...","(40.757242, -73.9801423, 0.0)",40.757242,-73.980142,0.0
4,Theater District,"(Theater District, Manhattan Community Board 5...","(40.7579801, -73.9855452, 0.0)",40.757980,-73.985545,0.0
5,Turtle Bay,"(Turtle Bay, Manhattan Community Board 6, Manh...","(40.7534666, -73.9688659, 0.0)",40.753467,-73.968866,0.0
6,Midtown East,"(Midtown East, Manhattan Community Board 5, Ma...","(40.7598219, -73.9724708, 0.0)",40.759822,-73.972471,0.0
7,Midtown,"(Midtown, Manhattan Community Board 5, Manhatt...","(40.7601085, -73.97816329087175, 0.0)",40.760109,-73.978163,0.0
8,Tudor City,"(Tudor City, Manhattan Community Board 6, Manh...","(40.7486231, -73.9713889, 0.0)",40.748623,-73.971389,0.0
9,Little Brazil,"(Little Brazil, Merton High Street, South Wimb...","(51.4157191, -0.191541, 0.0)",51.415719,-0.191541,0.0


### Here we drop na values in the NY table. We also drop rows with index 18 and 19 since they have located in California, which is inaccurate.

In [216]:
df1 = df1[df1['location'].notna()]

In [218]:
df1.drop([18,19])

KeyError: '[18 19] not found in axis'

### Here we will display maps of each neighborhood for both Toronto and NY.

In [219]:
map_Toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)
# adding markers to map
for latitude_toronto, longitude_toronto, borough, neighbourhood in zip(combined_toronto_data['Latitude'], combined_toronto_data['Longitude'], combined_toronto_data['Borough'], combined_toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude_toronto, longitude_toronto],
        radius=5,
        popup=label,
        color='red',
        fill=True
        ).add_to(map_Toronto)  
    
map_Toronto

In [220]:
map_ny = folium.Map(location=[latitude_ny, longitude_ny], zoom_start=11)
# adding markers to map
for latitude_ny, longitude_ny in zip(df1['latitude'], df1['longitude']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude_ny, longitude_ny],
        radius=5,
        popup=label,
        color='red',
        fill=True
        ).add_to(map_ny)  
    
map_ny

### Here we will use Foursquare to now find the venues, importing my client information and calling the API.

In [221]:
CLIENT_ID = 'O0OSRHMAQVOIRMXCENQ0GE2USNIS4UDLCI110WJPALIYDFYT' 
CLIENT_SECRET = 'W05D52IYHIRASPHRE5GIDIKX2VO0W4ZLYMGLOA1CUFQOHLC1'
VERSION = '20180604' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: O0OSRHMAQVOIRMXCENQ0GE2USNIS4UDLCI110WJPALIYDFYT
CLIENT_SECRET:W05D52IYHIRASPHRE5GIDIKX2VO0W4ZLYMGLOA1CUFQOHLC1


In [222]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius
            )
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

In [223]:
venues_in_toronto = getNearbyVenues(combined_toronto_data['Neighbourhood'], combined_toronto_data['Latitude'], combined_toronto_data['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [224]:
venues_in_ny = getNearbyVenues(df1['Name_of_the_neighborhood'], df1['latitude'], df1['longitude'])

Columbus Circle
Sutton Place
Rockefeller Center
Diamond District
Theater District
Turtle Bay
Midtown East
Midtown
Tudor City
Little Brazil
Times Square
Hudson Yards
Midtown West
Hell's Kitchen
Garment District
Herald Square
Koreatown


In [225]:
venues_in_toronto.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,Restaurant


In [226]:
venues_in_ny.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category
0,Columbus Circle,40.76808,-73.981897,Jazz at Lincoln Center,Jazz Club
1,Columbus Circle,40.76808,-73.981897,Marea,Seafood Restaurant
2,Columbus Circle,40.76808,-73.981897,NYRR RUNCenter,Community Center
3,Columbus Circle,40.76808,-73.981897,Champion Pizza,Pizza Place
4,Columbus Circle,40.76808,-73.981897,Mandarin Oriental,Hotel


### Here we group toronto's and NY's venues by "Neighbourhood" in order to get a better understanding of the business operations in each neighborhood.

In [227]:
venues_in_toronto.groupby('Neighbourhood').head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category
0,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,Bakery
1,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,Coffee Shop
2,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,Distribution Center
3,"Regent Park, Harbourfront",43.654260,-79.360636,Body Blitz Spa East,Spa
4,"Regent Park, Harbourfront",43.654260,-79.360636,Impact Kitchen,Restaurant
...,...,...,...,...,...
843,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Rorschach Brewing Co.,Brewery
844,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Leslieville Farmers Market,Farmers Market
845,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,The Sidekick,Comic Shop
846,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Chino Locos,Burrito Place


In [228]:
venues_in_ny.groupby('Neighbourhood').head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category
0,Columbus Circle,40.768080,-73.981897,Jazz at Lincoln Center,Jazz Club
1,Columbus Circle,40.768080,-73.981897,Marea,Seafood Restaurant
2,Columbus Circle,40.768080,-73.981897,NYRR RUNCenter,Community Center
3,Columbus Circle,40.768080,-73.981897,Champion Pizza,Pizza Place
4,Columbus Circle,40.768080,-73.981897,Mandarin Oriental,Hotel
...,...,...,...,...,...
446,Koreatown,40.747642,-73.986516,Baekjeong NYC (Kang Ho Dong Baekjeong),Korean Restaurant
447,Koreatown,40.747642,-73.986516,Jongro BBQ,Korean Restaurant
448,Koreatown,40.747642,-73.986516,Empire State Building,Building
449,Koreatown,40.747642,-73.986516,Lush,Cosmetics Shop


### We then group by "Venue Category" to show similarities between buisnesses and the neighborhoods they are located in.

In [229]:
venues_in_toronto.groupby('Venue Category').max()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue
Venue Category,,,,
Airport,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,Billy Bishop Toronto City Airport (YTZ) (Billy...
Airport Food Court,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,Billy Bishop Café
Airport Lounge,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,Porter Lounge
Airport Service,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,Porter Airlines Check-In Counter
Airport Terminal,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,Porter FBO Limited
...,...,...,...,...
Vegetarian / Vegan Restaurant,Stn A PO Boxes,43.657952,-79.373306,Veghed
Video Game Store,"University of Toronto, Harbord",43.662696,-79.400049,A & C Games
Vietnamese Restaurant,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,Sprout


In [230]:
venues_in_ny.groupby('Venue Category').max()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue
Venue Category,,,,
Accessories Store,Theater District,40.759822,-73.972471,Fossil Store
American Restaurant,Turtle Bay,40.760109,-73.968866,The Smith
Art Gallery,Rockefeller Center,40.760109,-73.972471,Sean Kelly Gallery
Art Museum,Midtown,40.768080,-73.978163,Museum of Modern Art (MoMA)
Arts & Crafts Store,Herald Square,40.753694,-73.987765,Mood Designer Fabrics
...,...,...,...,...
Vietnamese Restaurant,Tudor City,51.415719,-0.191541,Sai Gon Dep
Whisky Bar,Midtown West,40.764423,-73.992392,On the Rocks
Wine Bar,Turtle Bay,40.764423,-73.968866,Sofia Wine Bar


### We will get blank values here and remove. We will also move the Neighborhood column to the front.

In [231]:
toronto_venue_cat = pd.get_dummies(venues_in_toronto[['Venue Category']], prefix="", prefix_sep="")
toronto_venue_cat

,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
856,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
857,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
858,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [232]:
toronto_venue_cat['Neighbourhood'] = venues_in_toronto['Neighbourhood'] 

# moving neighborhood column to the first column
fixed_columns = [toronto_venue_cat.columns[-1]] + list(toronto_venue_cat.columns[:-1])
toronto_venue_cat = toronto_venue_cat[fixed_columns]

toronto_venue_cat.head()

,Neighbourhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [233]:
toronto_grouped = toronto_venue_cat.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.033333,...,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.041667
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.058824
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.066667,0.066667,0.133333,0.133333,0.133333,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.033333


In [234]:
ny_venue_cat = pd.get_dummies(venues_in_ny[['Venue Category']], prefix="", prefix_sep="")
ny_venue_cat

,Accessories Store,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Garage,Bagel Shop,Bakery,Bar,...,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
472,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
473,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
474,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [235]:
ny_venue_cat['Neighbourhood'] = venues_in_ny['Neighbourhood'] 

# moving neighborhood column to the first column
fixed_columns = [ny_venue_cat.columns[-1]] + list(ny_venue_cat.columns[:-1])
ny_venue_cat = ny_venue_cat[fixed_columns]

ny_venue_cat.head()

,Neighbourhood,Accessories Store,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Garage,Bagel Shop,Bakery,...,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store,Yoga Studio
0,Columbus Circle,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Columbus Circle,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Columbus Circle,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Columbus Circle,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Columbus Circle,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [236]:
ny_grouped = ny_venue_cat.groupby('Neighbourhood').mean().reset_index()
ny_grouped.head()

,Neighbourhood,Accessories Store,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Garage,Bagel Shop,Bakery,...,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store,Yoga Studio
0,Columbus Circle,0.0,0.000000,0.000000,0.033333,0.000000,0.0,0.0,0.000000,0.066667,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
1,Diamond District,0.0,0.066667,0.033333,0.000000,0.000000,0.0,0.0,0.000000,0.033333,...,0.066667,0.0,0.0,0.033333,0.033333,0.033333,0.000000,0.0,0.033333,0.0
2,Garment District,0.0,0.033333,0.033333,0.000000,0.033333,0.0,0.0,0.033333,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
3,Hell's Kitchen,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.033333,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.033333,0.1,0.000000,0.0
4,Herald Square,0.0,0.033333,0.000000,0.000000,0.033333,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0


### We will now return the most common venues for each city, which allows us to simplify the large amounts of data for stakeholders.

In [237]:
def return_most_common_venues_toronto(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [238]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_toronto = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_toronto['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted_toronto.iloc[ind, 1:] = return_most_common_venues_toronto(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_toronto.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Seafood Restaurant,Coffee Shop,Cocktail Bar,Beer Bar,Farmers Market,Bakery,Clothing Store,Tailor Shop,Jazz Club,Bistro
1,"Brockton, Parkdale Village, Exhibition Place",Café,Nightclub,Breakfast Spot,Coffee Shop,Performing Arts Venue,Burrito Place,Restaurant,Climbing Gym,Yoga Studio,Bar
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Auto Workshop,Skate Park,Smoke Shop,Burrito Place,Restaurant,Recording Studio,Butcher,Farmers Market,Fast Food Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Airport,Bar,Harbor / Marina,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Yoga Studio,Seafood Restaurant,Sandwich Place,Bubble Tea Shop,Ramen Restaurant,Poke Place,Modern European Restaurant


In [239]:
def return_most_common_venues_ny(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [240]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_ny = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_ny['Neighbourhood'] = ny_grouped['Neighbourhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighborhoods_venues_sorted_ny.iloc[ind, 1:] = return_most_common_venues_ny(ny_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_ny.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Columbus Circle,Jazz Club,French Restaurant,Spa,Juice Bar,Bakery,Kitchen Supply Store,Grocery Store,Food Court,Noodle House,Candy Store
1,Diamond District,American Restaurant,Sporting Goods Shop,Steakhouse,Toy / Game Store,Plaza,Department Store,Bookstore,Scenic Lookout,Salad Place,Fast Food Restaurant
2,Garment District,Hotel,Coffee Shop,Dance Studio,Theater,Burger Joint,Juice Bar,Gym,Lounge,Mediterranean Restaurant,Music Venue
3,Hell's Kitchen,Theater,Wine Bar,Mexican Restaurant,Hotel Bar,Bar,Men's Store,Spa,Mediterranean Restaurant,Pizza Place,Liquor Store
4,Herald Square,Clothing Store,Korean Restaurant,Burger Joint,Jewelry Store,Sandwich Place,Coffee Shop,Shoe Store,Salad Place,Snack Place,Indian Restaurant


### Finally we will build models using Kmeans and then visualize them again using folium

In [241]:
k_num_clusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans_toronto = KMeans(n_clusters=k_num_clusters, random_state=0).fit(toronto_grouped_clustering)
kmeans_toronto

KMeans(n_clusters=5, random_state=0)

In [242]:
kmeans.labels_[0:100]

array([3, 3, 3, 1, 3, 3, 4, 0, 0, 3, 3, 1, 3, 1, 2, 2, 0, 0], dtype=int32)

In [243]:
neighborhoods_venues_sorted_toronto.insert(0, 'Cluster Labels', kmeans_toronto.labels_)

In [244]:
k_num_clusters = 5

ny_grouped_clustering = ny_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans_ny = KMeans(n_clusters=k_num_clusters, random_state=0).fit(ny_grouped_clustering)
kmeans_ny

KMeans(n_clusters=5, random_state=0)

In [245]:
kmeans_ny.labels_[0:100]

array([2, 0, 2, 4, 2, 2, 3, 2, 0, 2, 4, 0, 1, 1, 2, 0], dtype=int32)

In [246]:
neighborhoods_venues_sorted_ny.insert(0, 'Cluster Labels', kmeans_ny.labels_)

In [247]:
toronto_merged = combined_toronto_data

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted_toronto.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Breakfast Spot,Café,Theater,Bakery,Historic Site,Distribution Center,Dessert Shop,Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Diner,College Cafeteria,Bar,Beer Bar,Smoothie Shop,Japanese Restaurant,Sandwich Place,Distribution Center
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Café,Theater,Coffee Shop,Hotel,Diner,Sandwich Place,Bookstore,Burger Joint,Ramen Restaurant,Burrito Place
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Gastropub,Café,Coffee Shop,Farmers Market,BBQ Joint,Movie Theater,Cocktail Bar,Middle Eastern Restaurant,Cosmetics Shop,Creperie
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Asian Restaurant,Trail,Neighborhood,Pub,Health Food Store,Creperie,Dog Run,Distribution Center,Discount Store,Diner


In [248]:
ny_merged = df1

ny_merged = ny_merged.join(neighborhoods_venues_sorted_ny.set_index('Neighbourhood'), on='Name_of_the_neighborhood')

ny_merged.head()

,Name_of_the_neighborhood,location,point,latitude,longitude,altitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Columbus Circle,"(Columbus Circle, Broadway, Columbus Circle, M...","(40.76808045, -73.98189744682783, 0.0)",40.768080,-73.981897,0.0,2.0,Jazz Club,French Restaurant,Spa,Juice Bar,Bakery,Kitchen Supply Store,Grocery Store,Food Court,Noodle House,Candy Store
1,Sutton Place,"(Sutton Place, Bull Lane, Sutton Green, Woking...","(51.27196705, -0.5510324033859777, 0.0)",51.271967,-0.551032,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Rockefeller Center,"(Rockefeller Center, 45, Rockefeller Plaza, Mi...","(40.75925755, -73.97995686116377, 0.0)",40.759258,-73.979957,0.0,0.0,American Restaurant,Steakhouse,French Restaurant,Food Truck,Plaza,Toy / Game Store,Scenic Lookout,Café,Gift Shop,Burrito Place
3,Diamond District,"(Diamond District, West 47th Street, Diamond D...","(40.757242, -73.9801423, 0.0)",40.757242,-73.980142,0.0,0.0,American Restaurant,Sporting Goods Shop,Steakhouse,Toy / Game Store,Plaza,Department Store,Bookstore,Scenic Lookout,Salad Place,Fast Food Restaurant
4,Theater District,"(Theater District, Manhattan Community Board 5...","(40.7579801, -73.9855452, 0.0)",40.757980,-73.985545,0.0,1.0,Theater,Hotel,Italian Restaurant,Plaza,Office,Performing Arts Venue,Chocolate Shop,Seafood Restaurant,Sushi Restaurant,Taco Place


In [249]:
toronto_merged_nonan = toronto_merged.dropna(subset=['Cluster Labels'])

In [250]:
ny_merged_nonan = ny_merged.dropna(subset=['Cluster Labels'])

### Visualizations

In [251]:
map_clusters_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k_num_clusters)
ys = [i + x + (i*x)**2 for i in range(k_num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_nonan['Latitude'], toronto_merged_nonan['Longitude'], toronto_merged_nonan['Neighbourhood'], toronto_merged_nonan['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster) +1) + '\n' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)]
        ).add_to(map_clusters_toronto)
        
map_clusters_toronto

In [252]:
map_clusters_ny = folium.Map(location=[latitude_ny, longitude_ny], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k_num_clusters)
ys = [i + x + (i*x)**2 for i in range(k_num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged_nonan['latitude'], ny_merged_nonan['longitude'], ny_merged_nonan['Name_of_the_neighborhood'], ny_merged_nonan['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster) +1) + '\n' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)]
        ).add_to(map_clusters_ny)
        
map_clusters_ny

### Examples of how to gather more information about each cluster

In [253]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 0, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,Coffee Shop,Park,Breakfast Spot,Café,Theater,Bakery,Historic Site,Distribution Center,Dessert Shop,Restaurant
4,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Diner,College Cafeteria,Bar,Beer Bar,Smoothie Shop,Japanese Restaurant,Sandwich Place,Distribution Center
24,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Café,Yoga Studio,Seafood Restaurant,Sandwich Place,Bubble Tea Shop,Ramen Restaurant,Poke Place,Modern European Restaurant
37,West Toronto,0,Bar,Asian Restaurant,Men's Store,Yoga Studio,Japanese Restaurant,Brewery,Italian Restaurant,Record Shop,Korean Restaurant,Ice Cream Shop
41,East Toronto,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Ice Cream Shop,Bakery,Cosmetics Shop,Pizza Place,Pub,Dessert Shop,Restaurant
47,East Toronto,0,Sandwich Place,Fast Food Restaurant,Liquor Store,Pub,Steakhouse,Italian Restaurant,Sushi Restaurant,Brewery,Board Shop,Fish & Chips Shop
67,Central Toronto,0,Park,Hotel,Breakfast Spot,Pizza Place,Sandwich Place,Gym / Fitness Center,Department Store,Food & Drink Shop,Discount Store,Diner
73,Central Toronto,0,Clothing Store,Coffee Shop,Yoga Studio,Health & Beauty Service,Salon / Barbershop,Spa,Restaurant,Sporting Goods Shop,Ice Cream Shop,Fast Food Restaurant
75,West Toronto,0,Gift Shop,Breakfast Spot,Eastern European Restaurant,Dessert Shop,Coffee Shop,Dog Run,Italian Restaurant,Bar,Movie Theater,Cuban Restaurant
79,Central Toronto,0,Dessert Shop,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Pizza Place,Sushi Restaurant,Gym,Diner,Indian Restaurant


In [254]:
ny_merged_nonan.loc[ny_merged_nonan['Cluster Labels'] == 0, ny_merged_nonan.columns[[1] + list(range(5, ny_merged_nonan.shape[1]))]]

,location,altitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"(Rockefeller Center, 45, Rockefeller Plaza, Mi...",0.0,0.0,American Restaurant,Steakhouse,French Restaurant,Food Truck,Plaza,Toy / Game Store,Scenic Lookout,Café,Gift Shop,Burrito Place
3,"(Diamond District, West 47th Street, Diamond D...",0.0,0.0,American Restaurant,Sporting Goods Shop,Steakhouse,Toy / Game Store,Plaza,Department Store,Bookstore,Scenic Lookout,Salad Place,Fast Food Restaurant
5,"(Turtle Bay, Manhattan Community Board 6, Manh...",0.0,0.0,Wine Bar,Japanese Restaurant,Sushi Restaurant,Karaoke Bar,Tailor Shop,Greek Restaurant,Food Truck,French Restaurant,Seafood Restaurant,Cheese Shop
7,"(Midtown, Manhattan Community Board 5, Manhatt...",0.0,0.0,American Restaurant,Plaza,Exhibit,Clothing Store,Jewelry Store,Gym,Outdoor Sculpture,Concert Hall,Food Truck,Scenic Lookout


## Results

In this project, we were successfully able to depict venues by neighborhood and cluster them based on Neighborhood. We were also able to visualize all the information into a usable format that will allow stakeholders to make a business decision. All location information that is displayed is accurate and it is evident that this would prove useful for potential business owners in the area. 

## Discussion

Both Toronto and Manhattan are diverse and include many different types of venues. It seems that any business proposal could be successful in these areas beacause of this diversity. Italian and asian restaurants are most popular in Toronto, while American restaurants are most popular in Manhattan. Restaurant business seem to be most popular in these areas and can prove to be successful in the future, but promote large amounts of competition. 

## Conclusion

The goal of this project was to offer business people the opportunity to view business in the areas of both cities with location data and visual aids to provide further details to them in order to assist in their decision. This project removes the need for constant research or physical travel by conveniently giving all the details in a well designed manner.

There is obvious diversity in the culture in both cities and the types of establishments they offer are fairly similar. For potential business owners, it seems that the possibility of success is extremely high.